# Reranking


In [12]:
import os
OPENAI_API_KEY=""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [13]:
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

### Helper Function to display nodes

In [14]:
import pandas as pd
from IPython.display import display, HTML
def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [15]:
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.indices.query.schema import QueryBundle
from llama_index.indices.postprocessor import LLMRerank


## Load Data

In [16]:
# data:folder name
documents = SimpleDirectoryReader("data").load_data()

## Build Index

In [18]:
# build index
index = VectorStoreIndex(nodes)


## Write your Promt and query the LLM

In [48]:
prompt = """ what is NLP
"""


In [49]:
query_engine = index.as_query_engine()
response = query_engine.query(prompt)
display(Markdown(f"<b>{response}</b>"))

<b>NLP stands for natural language processing. It is a field of study that involves the use of computer algorithms to analyze and understand human language.</b>

## Get K top similar nodes

In [29]:
# Configure retriever
retriever = VectorIndexRetriever(
  index=index,
  similarity_top_k=4,
)



In [50]:

# Retrieve top-k most similar nodes
query_bundle = QueryBundle(prompt) 
retrieved_nodes = retriever.retrieve(query_bundle)


#### No-rerank retrieved nodes

In [51]:
visualize_retrieved_nodes(retrieved_nodes)


,Score,Text
0,0.787408,Many great models are being introduced now a days that are h aving quite a heavy impact on the field on NLP.
1,0.785544,"Sentiment analysis is a natural language processing (NLP) technique used to determine whether data is positive, negative or neutral."
2,0.783839,NLTK library as of now gives an execution of the Po rter stemmer computation in the nltk.
3,0.782984,"Opinion Analysis is an NLP and data extract ion task that expects to acquire essayists felling communicated in certain or negative remarks, questions, and demands by breaking down an enormous number of archives."


## Rerank the nodes using "LLMRerank" and get K top reranked nodes

In [45]:
# Configure reranker
# choice_batch_size = number of nodes passed to LLMRerank to assign a score at one time
# top_n = number of nodes selected
reranker = LLMRerank(choice_batch_size=1, top_n=5)


In [52]:

# Rerank retrieved nodes
retrieved_nodes2 = reranker.postprocess_nodes(retrieved_nodes, query_bundle)

#### LLMRerank retrieved nodes

In [53]:
visualize_retrieved_nodes(retrieved_nodes2)


,Score,Text
0,10.0,"Sentiment analysis is a natural language processing (NLP) technique used to determine whether data is positive, negative or neutral."
1,10.0,"Opinion Analysis is an NLP and data extract ion task that expects to acquire essayists felling communicated in certain or negative remarks, questions, and demands by breaking down an enormous number of archives."
2,8.0,Many great models are being introduced now a days that are h aving quite a heavy impact on the field on NLP.


## Cohere Rerank

In [54]:
# Method1

import cohere
from llama_index.indices.postprocessor.cohere_rerank import CohereRerank

# Initialize the Cohere client
cohere_rerank  = CohereRerank(api_key='',top_n=3)

retrieved_nodes1 = cohere_rerank.postprocess_nodes(retrieved_nodes, query_bundle)

visualize_retrieved_nodes(retrieved_nodes1)
# Rerank the retrieved nodes
#results = client.rerank(model="rerank-english-v2.0", query=promt, documents=documents, top_n=3)


,Score,Text
0,0.833866,"Sentiment analysis is a natural language processing (NLP) technique used to determine whether data is positive, negative or neutral."
1,0.571886,NLTK library as of now gives an execution of the Po rter stemmer computation in the nltk.
2,0.526617,"Opinion Analysis is an NLP and data extract ion task that expects to acquire essayists felling communicated in certain or negative remarks, questions, and demands by breaking down an enormous number of archives."


In [114]:
# Method2

#compares the semantic information in the query and the initial search results
import cohere

# Initialize the Cohere client
client = cohere.Client(api_key='')


# Convert the retrieved nodes to a list of responses
responses = [node.text for node in retrieved_nodes]

# Rerank the responses
reranked_responses = client.rerank(
   model="rerank-english-v2.0",
   query=prompt,
   documents=responses,
   top_n=5
)
for i in range(2):
    print(i)
    print("relevance Score",reranked_responses[i].relevance_score)
    print("################")
    print(reranked_responses[i])
    print("---------------------------------------")
    print("---------------------------------------")
    print("---------------------------------------")
    print("---------------------------------------")
    print("---------------------------------------")
# Print the reranked responses


0
relevance Score 0.98884195
################
RerankResult<document['text']: International Journal of Trend in Scientific Researc h and Development (IJTSRD) @  www.ijtsrd.com  eISSN: 2456-6470 
@ IJTSRD | Unique Paper ID - IJTSRD23375   | Volume – 3 | Issue – 3 | Mar-Apr 2019 Page: 1371  2. Develop a method for automatic sentiment analysi s of 
Twitter messages. 
Opinion Mining and Sentiment Analysis, sentiment an alysis 
involves automatic analysis of opinions and emotive  lexicons 
expressed in a text. In the analysis of a text tona lity, it is 
considered that text information on the Internet is  divided 
into two classes: facts and opinions. The definitio n of an 
opinion is a key concept. Opinions are divided into  two 
types:  
1. Simple opinion.  
2. Comparison 
 
A simple opinion contains the statement of an author  about 
one entity. It can be stated directly: “I was pleas antly 
surprised with the furniture assembly quality”, or impl icitly: 
“After the treatment, my health b

## SentenceTransformerRerank

In [56]:
# Uses the cross-encoders from the sentence-transformer package to re-order nodes, and returns the top N nodes.
from llama_index.indices.postprocessor import SentenceTransformerRerank

# Initialize the SentenceTransformerRerank postprocessor
postprocessor = SentenceTransformerRerank(
   model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
   top_n=5
)

# Rerank the retrieved nodes
reranked_nodes4 = postprocessor.postprocess_nodes(nodes=retrieved_nodes, query_bundle=query_bundle)
visualize_retrieved_nodes(reranked_nodes4)

# Print the reranked nodes
for node in reranked_nodes4:
    print("SCORE:",node.score)
    print("#################")
    print(node.text)
    print("-------------------------------------")
    print("-------------------------------------")
    print("-------------------------------------")
    print("-------------------------------------")


,Score,Text
0,4.301733,"Sentiment analysis is a natural language processing (NLP) technique used to determine whether data is positive, negative or neutral."
1,1.241898,"Opinion Analysis is an NLP and data extract ion task that expects to acquire essayists felling communicated in certain or negative remarks, questions, and demands by breaking down an enormous number of archives."
2,-4.069395,Many great models are being introduced now a days that are h aving quite a heavy impact on the field on NLP.
3,-5.986939,NLTK library as of now gives an execution of the Po rter stemmer computation in the nltk.


SCORE: 4.301733
#################
Sentiment analysis is a natural  language  
processing (NLP) technique used to determine whether data is positive, negative or neutral.  

-------------------------------------
-------------------------------------
-------------------------------------
-------------------------------------
SCORE: 1.2418978
#################
Opinion Analysis is an NLP and data extract ion 
task that expects to acquire essayists felling 
communicated in certain or negative remarks, 
questions, and demands by breaking down an enormous  
number of archives.  

-------------------------------------
-------------------------------------
-------------------------------------
-------------------------------------
SCORE: -4.069395
#################
Many great models are being introduced  
now a days that are h aving quite a heavy impact on the field on NLP. 
-------------------------------------
-------------------------------------
-------------------------------------
-------

## BM25

In [116]:
from llama_index import (
    ServiceContext,
)
service_context = ServiceContext.from_defaults()
nodes = service_context.node_parser.get_nodes_from_documents(documents)

from llama_index.retrievers import BM25Retriever

# retireve the top 10 most similar nodes using bm25
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)


In [117]:
top_k_nodes = bm25_retriever._retrieve(query_bundle)

print(len(top_k_nodes))
# Print the retrieved documents and their scores
for node in top_k_nodes:
    print(f"Node: {node.node.get_content()}, Score: {node.score}")
    print("######################")

2
Node: International Journal of Trend in Scientific Researc h and Development (IJTSRD) @  www.ijtsrd.com  eISSN: 2456-6470 
@ IJTSRD | Unique Paper ID - IJTSRD23375   | Volume – 3 | Issue – 3 | Mar-Apr 2019 Page: 1371  2. Develop a method for automatic sentiment analysi s of 
Twitter messages. 
Opinion Mining and Sentiment Analysis, sentiment an alysis 
involves automatic analysis of opinions and emotive  lexicons 
expressed in a text. In the analysis of a text tona lity, it is 
considered that text information on the Internet is  divided 
into two classes: facts and opinions. The definitio n of an 
opinion is a key concept. Opinions are divided into  two 
types:  
1. Simple opinion.  
2. Comparison 
 
A simple opinion contains the statement of an author  about 
one entity. It can be stated directly: “I was pleas antly 
surprised with the furniture assembly quality”, or impl icitly: 
“After the treatment, my health became stronger”. I n both 
cases, a simple opinion usually has a posi

## KeywordNodePostprocessor

In [58]:
from llama_index.indices.postprocessor import KeywordNodePostprocessor

postprocessor = KeywordNodePostprocessor(
  required_keywords=["NLP"]
)

re=postprocessor.postprocess_nodes(retrieved_nodes)
visualize_retrieved_nodes(re)

print(len(re))
for i in re:
    print(i.text)
    print("######################")

,Score,Text
0,-4.069395,Many great models are being introduced now a days that are h aving quite a heavy impact on the field on NLP.
1,4.301733,"Sentiment analysis is a natural language processing (NLP) technique used to determine whether data is positive, negative or neutral."
2,1.241898,"Opinion Analysis is an NLP and data extract ion task that expects to acquire essayists felling communicated in certain or negative remarks, questions, and demands by breaking down an enormous number of archives."


3
Many great models are being introduced  
now a days that are h aving quite a heavy impact on the field on NLP. 
######################
Sentiment analysis is a natural  language  
processing (NLP) technique used to determine whether data is positive, negative or neutral.  

######################
Opinion Analysis is an NLP and data extract ion 
task that expects to acquire essayists felling 
communicated in certain or negative remarks, 
questions, and demands by breaking down an enormous  
number of archives.  

######################


## Custom Evaluation

In [156]:
textA = response
for i in retrieved_nodes:
    textB = i.text
    prompt=f"""you are a expert in text evaluation.
        I want you to compare two texts and return a relevance score of text A to text B.
        textA : {textA},
        textB : {textB},
        return the relevance score in json format
    """
    eval = query_engine.query(prompt)
    display(Markdown(f"<b>{eval}</b>"))

<b>{"relevance_score": 0.75}</b>

<b>{"relevance_score": 0.75}</b>

<b>{"relevance_score": 0.0}</b>

<b>{"relevance_score": 0.45}</b>

## evaluation techniques
### 1. Mean average precision (MAP)
### 2. Mean Reciprocal Rank (MRR)
### 3. Normalize Discounted Comulative Gain (NDCG)

In [ ]:
from sklearn.metrics import precision_recall_curve

y_true = [...]  # replace with your ground truth labels
y_scores = [...]  # replace with your predicted scores
precision, recall, _ = precision_recall_curve(y_true, y_scores)

from sklearn.metrics import average_precision_score

average_precision = average_precision_score(y_true, y_scores)

import torch
from torchmetrics import RetrievalMAP

map_metric = RetrievalMAP()
retrieval_scores = torch.tensor(y_scores)
relevant_docs = torch.tensor(y_true)
map_val = map_metric(retrieval_scores, relevant_docs)
